<a href="https://colab.research.google.com/github/kahinasassi/initial-text-preprocessing--arabic-dialect-/blob/master/dsaraby.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# translateration using DSARABY LABERARY

In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk import re
from nltk import defaultdict
import io
from nltk import  word_tokenize
from nltk import  sent_tokenize
from nltk.corpus import stopwords
import codecs
import csv
import pickle
from gensim.parsing.preprocessing import strip_short

nltk.download('punkt')

In [ ]:
import json,os,codecs
class DSAraby:
    def __init__(self):
        self.load_mapping()
        self.letters_to_ret = set()
    
    
    def load_mapping(self):

        dir_path = os.path.dirname(os.path.realpath('__file__'))
        self.en_to_ar = json.loads(open('/content/drive/My Drive/translateration/assets/mapping.manual.json', encoding='utf-8').read())
        self.NWORDS = {}
        word_counts = codecs.open('/content/drive/My Drive/translateration/assets/corpus.txt', encoding='utf-8').read().split("\n")
        for word_count in word_counts:
            if word_count:
                [word, n] = word_count.split()
                if word is None or n is None:
                    pass
                else:
                    self.NWORDS[word] = int(n)

    
    def transliterate(self,sentence, verbose=False):
        words = sentence.split()
        ret = []
        for word in words:
            candidates = list(self.transliterate_word(word))
            best_candidates = self.sort_by_frequency(candidates)
            if len(best_candidates) > 0:
                ret.append(self.sort_by_frequency(candidates)[0])
            else:
                ret.append(word)
        return ' '.join(ret)
    
    def transliterate_word(self,word):
        ret = self.transliterate_letter(word,'',True)
        self.letters_to_ret = set()
        return ret
    
    def transliterate_letter(self,letters, word, begin='start'):
        if len(letters) == 0:
            self.letters_to_ret.add(word)
            return
        
        if begin == 'start':
            table = self.en_to_ar['start']
        elif begin == 'other':
            table = self.en_to_ar['other']
        else :
            table = self.en_to_ar['end']
        max_key_len = len(max(list(table), key=len))
        for i in range(1, max_key_len + 1):
            l = letters[:i]
            if l in table:
                for ar in table[l]:
                    self.transliterate_letter(letters[i:], word + ar,'start')
        
        return self.letters_to_ret
    
    def sort_by_frequency(self,candidates): 
        return sorted(candidates, key=lambda k: self.NWORDS.get(k,0), reverse=True)

In [ ]:
p1 =  DSAraby()

In [ ]:
p1.transliterate(" bsh smahli nta zwiin nchlh b rabi saha")

'بسه صمىحلى نت زوين نشله ب ربي صاح'

In [ ]:
p1.transliterate(" sbah l khir alikom ")

'صباح ل خير اليكم'

In [ ]:
p1.transliterate(" wash rak tdir moh rak mlih")

'وش ريك تدير مه ريك ملح'

In [ ]:
p1.transliterate(" bsh smahli ")

'بسه صمىحلى'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
donnee=defaultdict()
data=open_pickle("/content/drive/My Drive/PROJET FIN ETUDE 2019 2020/ressources/donnee_phonetics.pickle")#charger dataset après phonétique grouping
donnee=defaultdict()
#mettre le corpus dans un dictionnaire pour facilier la manipulation 
for indice in list(data.index.values):
  donnee[indice]={'text':data['text'][indice],'label':data['label'][indice]}


In [ ]:
cpt=0
for indice in donnee.index :
        cpt=cpt+1
        liste=[]
        text=donnee[indice]['text']
        for word in tokenization(text):
          if(is_arabizi(word) == 1):
            print(word)
            tr=p1.transliterate(word)
            liste.append(tr)
          else: 
            liste.append(word)
        print(cpt," --- ",indice," ::   ==> ",liste)
        donnee[indice]={'text': liste, 'label' : donnee[indice]['label']}
             